## Что такое LSTM?

LSTM (Long Short-Term Memory) — это тип **рекуррентной нейронной сети (RNN)**, специально разработанный для работы с временными зависимостями. В отличие от стандартных RNN, LSTM лучше решает проблему **долговременной зависимости**, когда важные данные из начала временной последовательности могут теряться при передаче через множество шагов. Это делает LSTM особенно полезным для задач с последовательными данными, таких как временные ряды, текст и обработка последовательностей событий.

### Как работают стандартные RNN?

Чтобы понять, как работает LSTM, давайте коротко вспомним, как работают стандартные RNN:

- **Рекуррентные нейронные сети (RNN)** обрабатывают данные последовательностей, используя информацию из предыдущих временных шагов для предсказания следующего значения. Это делается за счет того, что RNN имеют "память" — их скрытые состояния (нейроны) зависят не только от текущего входа, но и от всех предыдущих входов.
  
- **Проблема с RNN**: Хотя стандартные RNN могут обрабатывать временные зависимости, у них есть недостаток — они плохо работают с длинными последовательностями. По мере увеличения количества шагов RNN теряет важные сигналы из ранних временных шагов (например, данные, поступившие 100 шагов назад). Это называется **проблемой исчезающего градиента**.

### Как LSTM решает проблему RNN?

LSTM — это улучшенная версия RNN, которая была специально разработана, чтобы справляться с долгосрочными зависимостями. Основное отличие LSTM от стандартных RNN — это способ хранения информации на протяжении длительного времени.

#### Основные компоненты LSTM:

- **Ячейка памяти (cell state)**: Главная особенность LSTM — это наличие ячейки памяти, которая может сохранять информацию на протяжении долгого времени. Состояние ячейки обновляется с помощью специальных механизмов — **входных, выходных и забывчивых "врат" (gates)**.
  
- **Врата (forget gate)**: Определяют, какую информацию из ячейки памяти можно забыть на текущем шаге. Это важно для того, чтобы модель могла "очищать" память от ненужной информации, чтобы не перегружать её.

- **Входные врата (input gate)**: Управляют тем, какую новую информацию можно добавить в ячейку памяти. Это позволяет LSTM обновлять память новыми данными, когда это нужно.

- **Выходные врата (output gate)**: Решают, какую информацию из ячейки памяти использовать для текущего выхода и передачи на следующий шаг.

Благодаря этим компонентам, LSTM может эффективно обучаться на данных с длинными зависимостями, где важная информация из начала последовательности может быть использована для предсказаний на более поздних шагах.

---

### Как LSTM применяется к задачам временных рядов?

Когда мы работаем с **временными рядами** (например, с данными о продажах магазинов за несколько дней), важно учитывать зависимости между прошлыми событиями и текущим предсказанием. В нашем случае:

- **Входные данные**: Это последовательности временных шагов, например, за последние 14 дней. Для каждого дня у нас есть набор признаков, таких как количество продаж, наличие акций, конкуренты и т.д.
  
- **LSTM-слой**: Каждый слой LSTM будет обрабатывать временные зависимости в данных. В отличие от стандартных полносвязных слоев, LSTM сохраняет информацию о последовательности событий и может использовать эту информацию для улучшения предсказания.
  
- **Выход**: На выходе LSTM предсказывает продажи на следующий день, основываясь на данных за предыдущие 14 дней. Модель может обновлять свои предсказания, обучаясь на каждой новой последовательности.

---

### Почему LSTM лучше, чем обычные нейронные сети для временных рядов?

В задачах временных рядов важна не только текущая информация, но и история событий. Простая полносвязная нейронная сеть не может учесть взаимосвязи между предыдущими и текущими данными. В то же время LSTM:

1. **Хранит информацию о последовательности**: Ячейка памяти и "врата" позволяют модели сохранять важные данные на протяжении всей последовательности.
   
2. **Предсказывает будущее на основе прошлого**: LSTM эффективно использует историю данных для предсказания будущих значений, что особенно полезно в задачах прогнозирования продаж, спроса, цен и других временных рядов.

# Материал по созданию и обучению модели LSTM для прогнозирования продаж с использованием временных рядов

В этом уроке мы пошагово разберем, как создать модель LSTM для прогнозирования продаж на основе временных рядов. Модель будет обучаться на первых 5% данных и тестироваться на разных сегментах данных с шагом в 5%.

## Разбор кода

### Шаг 0: Добавление датасета в блокнот

Загрузите файлы `store.csv` и `train.csv` в файлы блокнота.

Скачать датасет можно тут: https://drive.google.com/drive/folders/1B41RvyVuFayZRFcHuJwGO_ltdrQZCRWP?usp=sharing

Источник датасета и информация о нём: https://www.kaggle.com/c/rossmann-store-sales/data

### Шаг 1: Импорт необходимых библиотек

```python
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from tensorflow.keras import layers
```

**Описание:**
- **pandas**: Библиотека для работы с табличными данными (DataFrame).
- **numpy**: Используется для работы с массивами данных и числовыми вычислениями.
- **tensorflow**: Библиотека для построения и обучения нейронных сетей. В частности, мы используем её для создания LSTM модели.
- **sklearn.preprocessing.StandardScaler**: Стандартный скейлер, который используется для нормализации данных.
- **matplotlib.pyplot**: Библиотека для визуализации графиков.

---

### Шаг 2: Загрузка данных и их предобработка

```python
# Загрузка данных
train = pd.read_csv('train.csv', low_memory=False)
store = pd.read_csv('store.csv')

# Объединение данных по идентификатору магазина (Store)
data = pd.merge(train, store, on='Store', how='left')
```

**Описание:**
- **train.csv** содержит данные о продажах магазинов за определенные дни.
- **store.csv** содержит дополнительную информацию о магазинах (например, расстояние до конкурентов).
- Мы объединяем данные по столбцу `Store`, чтобы получить всю необходимую информацию в одном наборе данных.

---

### Шаг 3: Предобработка данных

```python
data['Date'] = pd.to_datetime(data['Date'])
data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month
data['Day'] = data['Date'].dt.day
data = data.drop(columns=['Date'])

data = data[data['Open'] == 1]
data = data.drop(columns=['Open', 'Customers'])

data['CompetitionDistance'] = data['CompetitionDistance'].fillna(data['CompetitionDistance'].median())
data['PromoInterval'], _ = pd.factorize(data['PromoInterval'])
data = data.fillna(0)

data = pd.get_dummies(data, columns=['DayOfWeek', 'StoreType', 'Assortment', 'StateHoliday'], drop_first=True)

for column in data.select_dtypes(include=['bool']).columns:
    data[column] = data[column].astype(int)
```

**Описание:**
- **Преобразование даты**: Мы извлекаем признаки даты (год, месяц, день) и удаляем сам столбец даты.
- **Удаление ненужных строк и столбцов**: Убираем строки, где магазин был закрыт (`Open == 0`), и удаляем ненужные столбцы, такие как `Customers` и `Open`.
- **Обработка пропущенных значений**: Заполняем пропуски в данных о расстоянии до конкурентов (`CompetitionDistance`) медианным значением и преобразуем строковые значения в числовые в столбце `PromoInterval`.
- **One-hot encoding**: Преобразуем категориальные переменные в числовые с помощью метода `get_dummies()`.
- **Преобразование булевых значений**: Преобразуем логические значения в целочисленные (0 или 1).

---

### Шаг 4: Разбиение данных и подготовка последовательностей

```python
train_size = int(len(data) * 0.05)  # 5% данных для обучения
test_size = int(len(data) * 0.05)  # Тестирование будет происходить на каждых 5% данных

train_data = data.iloc[:train_size]
test_data = data.iloc[train_size:]  # Остальные 95% для тестирования (каждые 5% будем тестировать по очереди)

def create_sequences(X, y, time_steps=14):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X[i:(i + time_steps)])
        ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)

X_train = train_data.drop(columns=['Sales']).values
y_train = train_data['Sales'].values

X_test = test_data.drop(columns=['Sales']).values
y_test = test_data['Sales'].values
```

**Описание:**
- **Отбор данных**: Мы берем первые 5% данных для обучения модели, а остальные 95% данных будем использовать для тестирования.
- **Функция создания последовательностей**: Для LSTM нам нужны последовательности данных, поэтому мы создаем временные окна длиной 14 дней (2 недели).
- **Разделение на признаки и целевую переменную**: Мы отделяем столбец с продажами (`Sales`) от остальных признаков для обучения модели.

---

### Шаг 5: Нормализация данных и создание последовательностей

```python
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

time_steps = 14
X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train, time_steps)
```

**Описание:**
- Мы нормализуем данные с помощью **StandardScaler** для того, чтобы все признаки имели одинаковый масштаб. Это важно для ускорения сходимости модели и повышения её точности.
- Затем мы создаем последовательности данных длиной 14 дней для подачи в LSTM.

---

### Шаг 6: Создание LSTM модели

```python
model = tf.keras.Sequential()
model.add(layers.Input(shape=(X_train_seq.shape[1], X_train_seq.shape[2])))
model.add(layers.LSTM(64, activation='relu', return_sequences=True))
model.add(layers.LSTM(32, activation='relu'))
model.add(layers.Dense(1))

model.compile(optimizer='adam', loss='mse')
```

**Описание:**
- **Последовательная модель**: Мы используем **Sequential** модель, в которую поочередно добавляем слои.
- **Первый слой LSTM**: Обрабатывает последовательность длиной 14 дней (две недели). Каждый временной шаг содержит набор признаков.
- **Второй слой LSTM**: Уменьшает количество нейронов до 32 и возвращает только последнее значение последовательности (предсказание на следующий день).
- **Полносвязный слой (Dense)**: Этот слой отвечает за финальный вывод одного значения — предсказания продаж. Мы предсказываем продажи только на 1 день, на основе признаков за 14 предыдущих дней.

---

### Шаг 7: Обучение модели

```python
history = model.fit(X_train_seq, y_train_seq, epochs=50, batch_size=64, validation_split=0.1, verbose=1)
```

**Описание:**
- Модель обучается на 50 эпохах с использованием данных, разделенных на обучающую и валидационную выборки. Мы используем 10% данных для валидации.

---

### Шаг 8: Функция для тестирования предсказаний с поддержкой реальных признаков

```python
def plot_real_data_predictions(X_test_scaled, y_test, time_steps, model, test_size, total_data_len):
    current_start = 0
    plt.figure(figsize=(12, 10))

    while current_start < len(X_test_scaled):
        end_idx = current_start + test_size
        if end_idx >= len(X_test_scaled):
            end_idx = len(X_test_scaled)

        X_test_seq, y_test_seq = create_sequences(X_test_scaled[current_start:end_idx], y_test[current_start:end_idx], time_steps)
        y_test_pred = model.predict(X_test_seq)

        plt.figure(figsize=(12, 6))
        plt.plot(y_test_seq, label='Настоящие значения')
        plt.plot(y_test_pred, label='Предсказанные значения')
        plt.legend()
        plt.title(f'Предсказания с {current_start} по {end_idx}')
        plt.show()

        current_start += test_size
```

**Описание:**
- Эта функция позволяет предсказывать и визуализировать результаты модели на тестовых данных с шагом в 5% данных.
- Мы строим графики, сравнивая реальные значения продаж с предсказанными значениями, чтобы видеть, насколько хорошо модель справляется.

---

### Шаг 9: Вызов функции для тестирования и визуализации

```python
total_data_len = len(data)
plot_real_data_predictions(X_test_scaled, y_test, time_steps, model, test_size, total_data_len)
```

**Описание:**
- Мы запускаем функцию, которая предсказывает результаты с интервалом в 5% от общего объема данных и выводит их на график для оценки качества модели.

# Задачи для самостоятельного выполнения

### 1. **Изменить количество временных шагов (timestamps)**
   
**Описание задачи**: В данный момент модель обучается на последовательностях длиной 14 дней (2 недели). Попробуйте изменить количество временных шагов на другие значения (например, 7, 30 или 60 дней) и проанализируйте, как это влияет на точность прогнозов.

**Цель**:
- Изучить, как длина последовательности (количество шагов назад) влияет на предсказания.
- Найти оптимальное количество временных шагов для задачи прогнозирования продаж.

**Задание**:
- Измените параметр `time_steps` в функции `create_sequences()`.
- Обучите модель и сравните результаты по MAE и графикам.

---

### 2. **Изменить количество нейронов в слоях LSTM**

**Описание задачи**: Мы использовали два слоя LSTM с 64 и 32 нейронами соответственно. Попробуйте изменить количество нейронов в каждом слое (например, увеличьте или уменьшите в два раза) и проанализируйте влияние этого на качество модели.

**Цель**:
- Найти оптимальное количество нейронов для слоев LSTM.
- Понять, как сложность модели влияет на ее обучаемость и качество предсказаний.

**Задание**:
- Измените количество нейронов в слоях LSTM.
- Обучите модель и проанализируйте, как это повлияло на скорость обучения и качество предсказаний.

---

### 3. **Добавить слой Dropout для борьбы с переобучением**

**Описание задачи**: Модель может переобучаться на обучающих данных, особенно если она сложная. Один из способов борьбы с переобучением — это добавление слоя **Dropout**, который случайным образом отключает некоторые нейроны во время обучения, предотвращая модель от заучивания специфичных шаблонов.

**Цель**:
- Улучшить обобщающую способность модели и снизить вероятность переобучения.

**Задание**:
- Добавьте слой `Dropout` после каждого слоя LSTM с вероятностью 0.2 или 0.3.
- Обучите модель и проверьте, как это влияет на результаты на тестовых данных.

---

### 4. **Добавить больше слоев LSTM**

**Описание задачи**: Попробуйте добавить больше слоев LSTM. Например, вместо двух слоев добавьте три или четыре, чтобы модель могла лучше обрабатывать сложные зависимости во временных рядах.

**Цель**:
- Понять, как добавление дополнительных слоев LSTM может улучшить (или ухудшить) предсказания.
- Проанализировать, как это влияет на вычислительную сложность и время обучения.

**Задание**:
- Добавьте еще один или два слоя LSTM.
- Проверьте, как это изменение повлияло на результаты.

---

### 5. **Изменить функцию активации на других слоях**

**Описание задачи**: Мы используем функцию активации `ReLU` (Rectified Linear Unit) в слоях LSTM. Попробуйте заменить её на другие функции активации, такие как `tanh` или `sigmoid`, и посмотрите, как это влияет на обучение модели.

**Цель**:
- Понять, как функция активации влияет на результаты и обучаемость модели.

**Задание**:
- Измените функцию активации в слоях LSTM.
- Обучите модель и сравните результаты с предыдущими экспериментами.

---

### 6. **Использовать больше данных для обучения**

**Описание задачи**: В данный момент мы используем только 5% данных для обучения. Попробуйте увеличить этот процент до 10% или 20% и посмотрите, как это влияет на качество предсказаний.

**Цель**:
- Изучить, как увеличение объема данных для обучения влияет на точность и обобщающую способность модели.

**Задание**:
- Увеличьте количество данных, например, до 10% или 20%, и обучите модель на большем объеме данных.
- Проверьте, как это влияет на метрики модели.

---

### 7. **Изменить размер батча**

**Описание задачи**: В текущем коде размер батча установлен на 64. Попробуйте изменить этот параметр на более маленькие (например, 32) или большие значения (например, 128), чтобы изучить, как это влияет на время обучения и точность модели.

**Цель**:
- Найти оптимальный размер батча для ускорения обучения и улучшения качества предсказаний.

**Задание**:
- Измените параметр `batch_size` на 32 или 128 и посмотрите, как это повлияет на результаты.
- Сравните результаты при разных размерах батча.

---

Эти задачи помогут вам лучше понять, как архитектура модели и гиперпараметры влияют на её обучение и точность. Экспериментируя с архитектурой LSTM, количеством временных шагов и другими параметрами, вы сможете улучшить производительность модели и лучше понять, как LSTM работает с временными рядами.